<a href="https://colab.research.google.com/github/Nagmashaik123/MedConcepts/blob/main/TweetFormatted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import json
import csv
import collections

dataList = []
def writeTweetData():
    dyfunctionfile = open('MedicalDyfunctions.json')
    jsonList = json.load(dyfunctionfile)
    print(jsonList)
    global dataList
    fileName = 'MedicalConcepts'
    with open('%s.csv' % (fileName), 'w') as file:
        w = csv.writer(file)
        w.writerow(['CUID', 'Concept', 'AuthorID', 'TweetText', 'CreatedDate'])
        for jsonData in jsonList:
            for jsonKey, jsonValues in jsonData.items():
                dataList = jsonData['tweets']['data']
                for data in dataList:
                    for key, value in data.items():
                        authorID  = data['author_id']
                        textData  = data['text']
                        createdDt = data['created_at']
                        w.writerow([jsonData['cui'], jsonData['concepts'], authorID, textData, createdDt])


writeTweetData()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [14]:
import pandas as pd
dataFrame = pd.read_csv(r"MedicalConcepts.csv")

#df = pd.DataFrame(dataFrame, columns = ['CUID', 'Concept', 'AuthorID', 'TweetText', 'CreatedDate'])
#for i in range(len(df)):

df = dataFrame.drop_duplicates(
  subset = ['CUID', 'AuthorID', 'TweetText','CreatedDate'],
  keep = 'last').reset_index(drop = True)

df



,CUID,Concept,AuthorID,TweetText,CreatedDate
0,C0003123,Anorexia,1399574560601673728,switch the v and the r and even Instagram ads ...,2021-06-25T23:44:49.000Z
1,C0003123,Anorexia,1347182874986950658,here use me as thinspo 🦋\n\n#thinspo #proana #...,2021-06-25T23:41:26.000Z
2,C0003123,Anorexia,14519812,😲😄👉👍👍👍Mujer que sufría de anorexia se cura des...,2021-06-25T23:01:03.000Z
3,C0003123,Anorexia,1345332202847019008,RT @starvingd0lly: ♡ some of my favorite thins...,2021-06-25T21:42:02.000Z
4,C0003123,Anorexia,902627303913611268,Obésité et anorexie monte en flèche dans #ndg ...,2021-06-25T21:32:26.000Z
...,...,...,...,...,...
24869,C0231230,Fatigability,202374786,#fatigability,2012-12-08T13:50:11.000Z
24870,C0231230,Fatigability,127451843,Sakit sa kamay! Oh shet! -_- #Fatigability,2012-08-03T07:26:10.000Z
24871,C0231230,Fatigability,262514041,What Rhymes With Fatigability... http://t.co/u...,2012-02-22T19:43:52.000Z
24872,C0231230,Fatigability,81819178,#fatigability,2012-01-25T11:54:09.000Z


In [ ]:
for tweet in df["TweetText"]:
  

In [15]:
import re

# Reading emoji patterns which contains emoticons , symbols & pictographs, transport & map symbols , flags
emojisCollection = re.compile("["
                       u"\U0001F600-\U0001F64F"  
                       u"\U0001F300-\U0001F5FF"  
                       u"\U0001F680-\U0001F6FF"  
                       u"\U0001F1E0-\U0001F1FF"  
                       u"\U00002702-\U000027B0"
                       u"\U000024C2-\U0001F251"
                       "]+", flags=re.UNICODE)
df['CleanedTweet'] = (df['TweetText'].str.lower().str.replace(r'[^\w\s]+', '').str.replace(emojisCollection, '').str.strip())

In [16]:
dataFrame=df[['CUID','Concept','AuthorID','TweetText','CleanedTweet','CreatedDate']]
dataFrame

,CUID,Concept,AuthorID,TweetText,CleanedTweet,CreatedDate
0,C0003123,Anorexia,1399574560601673728,switch the v and the r and even Instagram ads ...,switch the v and the r and even instagram ads ...,2021-06-25T23:44:49.000Z
1,C0003123,Anorexia,1347182874986950658,here use me as thinspo 🦋\n\n#thinspo #proana #...,here use me as thinspo \n\nthinspo proana anor...,2021-06-25T23:41:26.000Z
2,C0003123,Anorexia,14519812,😲😄👉👍👍👍Mujer que sufría de anorexia se cura des...,mujer que sufría de anorexia se cura después d...,2021-06-25T23:01:03.000Z
3,C0003123,Anorexia,1345332202847019008,RT @starvingd0lly: ♡ some of my favorite thins...,rt starvingd0lly some of my favorite thinspos...,2021-06-25T21:42:02.000Z
4,C0003123,Anorexia,902627303913611268,Obésité et anorexie monte en flèche dans #ndg ...,obésité et anorexie monte en flèche dans ndg o...,2021-06-25T21:32:26.000Z
...,...,...,...,...,...,...
24869,C0231230,Fatigability,202374786,#fatigability,fatigability,2012-12-08T13:50:11.000Z
24870,C0231230,Fatigability,127451843,Sakit sa kamay! Oh shet! -_- #Fatigability,sakit sa kamay oh shet _ fatigability,2012-08-03T07:26:10.000Z
24871,C0231230,Fatigability,262514041,What Rhymes With Fatigability... http://t.co/u...,what rhymes with fatigability httptcouujssyks ...,2012-02-22T19:43:52.000Z
24872,C0231230,Fatigability,81819178,#fatigability,fatigability,2012-01-25T11:54:09.000Z
